<a href="https://colab.research.google.com/github/Skystapper/ooba-sillytavern-chat-history-convert/blob/main/ooba-to-sillytavern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  **A simple Chat History Converter 🚀**

This interactive Colab notebook is your go-to tool for converting chat histories from ooba's .json format to sillytavern's .jsonl format . An effort to convert my too lengthy Oobabooga AI chat history into a structure compatible with Sillytavern because I couldn't find any tool to do that properly

### **How to Use:**

1. **Enter Your Details:**
    - Provide the character names in your chat history.
    - Specify the input file path or choose to upload it during runtime.
    - Set your desired output file name.

2. **Run the Code Cell:**
    - Execute code cell by pressing Shift + Enter or just clicking on that play button thing.
    - Follow the prompts to upload files if needed.

3. **Download Your Output:**
    - Once the conversion is complete, click the "Download Output File" button to get your final result.

4. **Explore and Share:**
    - Your converted chat history is ready for use! Explore the structure or share it as needed.

You are free to edit the code structure of this notebook if you need extra things in your chat history, I've kept it simple just as per my need 🌟



In [ ]:
from google.colab import files
import json
from datetime import datetime
import ipywidgets as widgets

# @title **Enter Details**
# @markdown Please enter the required details.

# @markdown **Character Names**
user_name = "Your name in the chat history"  # @param {type: "string"}
ai_name = "AI character's name in chat history"  # @param {type: "string"}

# @markdown **Input File**
upload_input_file = False  # Set to True if you want to upload a file
input_file_path = None

if not upload_input_file:
    try:
        input_file_path = "path of the input file if it's already uploaded in Colab storage"  # @param {type: "string"}
        with open(input_file_path, 'r'):
            pass  # Check if the file exists and can be opened
    except FileNotFoundError:
        print("File not found. You will be prompted to upload the file during runtime.")
        uploaded = files.upload()
        input_file_path = list(uploaded.keys())[0]

# @markdown **Output File Name**
output_file_name = ""  # @param {type: "string"}

# Part 1: Extracts the chat from the visible section of your ooba chat history and stores it in another JSON file
def extract_visible_chat(input_file_path, output_file_path):
    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
        data = json.load(input_file)
        visible_data = data.get('visible', [])
        json.dump(visible_data, output_file)

# Part 2: Reads the generated JSON file and converts it into a format that matches the structure of Sillytavern chat history
def generate_message(name, is_user, message, extra=None):
    if not message.strip():
        return None  # Skip entries with empty messages

    current_time = datetime.now().strftime("%B %d, %Y %I:%M%p")
    output = {
        "name": name,
        "is_user": is_user,
        "send_date": current_time,
        "mes": message,
        "extra": extra if extra else {}
    }
    if is_user:
        output["force_avatar"] = "User Avatars/user-default.png"
    return json.dumps(output, indent=4)

def process_input_file(input_messages, output_file_path, user_name, ai_name):
    with open(output_file_path, 'w') as output_file:
        for messages in input_messages:
            user_message = messages[0]
            ai_message = messages[1]

            user_output = generate_message(user_name, True, user_message, {})
            ai_output = generate_message(ai_name, False, ai_message, {
                "gen_started": "2023-11-09T06:12:56.823Z",
                "gen_finished": "2023-11-09T06:13:23.457Z",
                "swipe_id": 0,
                "swipes": [ai_message],
                "swipe_info": [
                    {
                        "send_date": "November 9, 2023 11:43am",
                        "gen_started": "2023-11-09T06:12:56.823Z",
                        "gen_finished": "2023-11-09T06:13:23.457Z",
                        "extra": {
                            "api": "textgenerationwebui",
                            "model": "TheBloke_echidna-tiefigther-25-GPTQ"
                        }
                    }
                ]
            })

            if user_output:
                output_file.write(user_output + '\n\n')
            if ai_output:
                output_file.write(ai_output + '\n\n')

# Part 3: Generates a final output file with .jsonl extension which can be used in sillytavern
def convert_json_to_jsonl(input_file_path, output_file_path, user_name, ai_name):
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        data = ""
        for line in infile:
            data += line.strip()
            try:
                json_data = json.loads(data)
                formatted_json = json.dumps(json_data, separators=(',', ':')) + '\n'
                outfile.write(formatted_json)
                data = ""
            except json.JSONDecodeError:
                pass

    # Modification Part: Modifies the first line of the generated .jsonl file
    with open(output_file_path, 'r') as file:
        lines = file.readlines()

    if lines:
        first_line = json.loads(lines[0])
        first_line['is_system'] = False
        first_line['send_date'] = "November 9, 2023 11:40am"
        first_line['extra'] = {}
        first_line = {k: first_line[k] for k in ['name', 'is_user', 'is_system', 'send_date', 'mes', 'extra']}

        lines.insert(0, json.dumps(first_line) + '\n')

        with open(output_file_path, 'w') as file:
            file.writelines(lines)

        # Shift all lines by one position
        for i in range(len(lines)-1, 0, -1):
            lines[i] = lines[i-1]

        # Add the extra line at the beginning
        extra_line = f'{{"user_name":"{user_name}","character_name":"{ai_name}","create_date":"{datetime.now().strftime("%Y-%m-%d@%Hh%Mm%Ss")}","chat_metadata":{{"note_prompt":"","note_interval":1,"note_position":1,"note_depth":4}}}}\n'
        lines[0] = extra_line

        with open(output_file_path, 'w') as file:
            file.writelines(lines)

# Process the input file
extract_visible_chat(input_file_path, '/content/output.json')
input_messages = json.load(open('/content/output.json', 'r'))
output_file_path = f"/content/{output_file_name}.jsonl"
process_input_file(input_messages, '/content/output.json', user_name, ai_name)
convert_json_to_jsonl('/content/output.json', output_file_path, user_name, ai_name)
print(f"Conversion completed. Output file: {output_file_path}")

# Button to trigger download
download_button = widgets.Button(description="Download Output File")
output_file_path_button = widgets.Output()

def on_download_button_click(b):
    with output_file_path_button:
         files.download(output_file_path)

download_button.on_click(on_download_button_click)
display(download_button, output_file_path_button)
